# 3-tangle

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/misc/entangle-3tangle.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi


## Pure state

Hyperdeterminant [wiki-link](https://en.wikipedia.org/wiki/Hyperdeterminant) of a tensor $x\in\mathbb{C}^{2\times 2\times 2}$ is defined as

$$ \mathrm{Det}(x)=\sum_{ijkl\in[8]}c_{ijkl}x_{i}x_{j}x_{k}x_{l}\;:\mathbb{C}^{8}\to\mathbb{C} $$

For simplicity, the notation above treats the tensor as a vector in $\mathbb{C}^{8}$, and the constant coefficents $c_{ijkl}$ can be found in [wiki-link](https://en.wikipedia.org/wiki/Hyperdeterminant). The hyperdeterminant is a polynomial of degree 4 in the tensor elements. The 3-tangle of a pure state $|\psi\rangle\in\mathbb{C}^{2}\otimes\mathbb{C}^{2}\otimes\mathbb{C}^{2}$ is defined as

$$ \tau\left(|\psi\rangle\right)=4\left|\mathrm{Det}\left(|\psi\rangle\right)\right| $$

In [ ]:
psi = numqi.state.GHZ(n=3)
print('3tangle(GHZ) =', numqi.entangle.get_3tangle_pure(psi))

In [ ]:
psi = numqi.random.rand_haar_state(8)
print('3tangle(random) =', numqi.entangle.get_3tangle_pure(psi))

In [ ]:
psiA = numqi.random.rand_haar_state(2)
psiBC = numqi.random.rand_haar_state(4)
psi = np.kron(psiA, psiBC)
print('3tangle(product state) =', numqi.entangle.get_3tangle_pure(psi))

## Mixed state

3-tangle of a mixed state $\rho\in\mathbb{C}^{8\times 8}$ is defined via the convex roof extension of the pure state 3-tangle

$$ \begin{align*}
    \tau(\rho)&=\min_{\left\{ p_{\alpha},|\psi_{\alpha}\rangle\right\} }\sum_{\alpha}p_{\alpha}\tau\left(|\psi_{\alpha}\rangle\right)\\
    &=\min_{\left\{ |\tilde{\psi}_{\alpha}\rangle\right\} }\sum_{\alpha}\frac{\tau\left(|\tilde{\psi}_{\alpha}\rangle\right)}{\langle\tilde{\psi}_{\alpha}|\tilde{\psi}_{\alpha}\rangle}
\end{align*} $$

where the minimization is over all possible pure state decompositions of the mixed state $\rho=\sum_{\alpha}p_{\alpha}|\psi_{\alpha}\rangle\langle\psi_{\alpha}|$.

Below, we evaluate the 3-tangle from the maximally mixed state (separable) to GHZ state (entangled) via gradient based optimization [arxiv-link](https://arxiv.org/abs/2406.19683).

In [ ]:
psi_ghz = numqi.state.GHZ(n=3)
dm_target = psi_ghz.reshape(8,1) * psi_ghz.conj()
alpha_list = np.linspace(0, 1, 100)

# about 3 minutes
model = numqi.entangle.ThreeTangleModel(num_term=4*8)
ret_opt = []
for alpha_i in tqdm(alpha_list):
    model.set_density_matrix(numqi.entangle.hf_interpolate_dm(dm_target, alpha=alpha_i))
    ret_opt.append(numqi.optimize.minimize(model, 'uniform', num_repeat=3, tol=1e-8, print_every_round=0).fun)
ret_opt = np.array(ret_opt)


One can find that for the segment $\alpha\in[0,\alpha^*\approx 0.7]$, 3-tangle is almost zero, which indicates the state is separable. For larger $\alpha$, the 3-tangle increases, and goes to 1 for GHZ state. 

TODO: the boundary $\alpha^*=0.7$ drawn in the plot is not theorectical proved boundary (need to check reference), and I just plot it for illustration purpose.

TODO: the optimization has convergence issue for those separable density matrix, whose 3-tangle should be exactly zero but cannot be optimized below $10^{-4}$.

In [ ]:
fig,(ax0,ax1) = plt.subplots(1, 2, figsize=(9,4))
for ax in [ax0,ax1]:
    ax.plot(alpha_list, ret_opt, 'x', label='manifold-opt')
    ax.set_xlabel('alpha')
    ax.set_ylabel('3-tangle')
    ax.axvline(0.7, color='red') #I don't know whether alpha=0.7 is the boundary, I just guess so
ax1.set_yscale('log')
fig.tight_layout()